<a href="https://colab.research.google.com/github/syedmahmoodiagents/Stategraph/blob/main/Typing_Pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydantic --q

In [ ]:
!pip install pydantic[email] --q

# Pydantic used mainly for type validation and data validation

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import TypeDict, List, Dict, Optional, Annotated

In [ ]:
# from pydantic import BaseModel

# BaseModel

In [ ]:
from pydantic import BaseModel

In [ ]:
class person2(BaseModel):
    name: str
    age: int = 20
    weight: float

In [ ]:
r = person2(name='mahmood', weight=75.3, age=23)

In [ ]:
r.name, r.age, r.weight

('mahmood', 23, 75.3)

In [ ]:
class Abcd(TypedDict):
    question: str
    answer: str
    critique: str
    retry_count: int

In [ ]:
 print(Abcd.__annotations__)

# Using dictionary

In [ ]:
# !pip install pydantic[email] --q

In [ ]:
class person(BaseModel):
    name: str = Field(max_length=8)
    age: int = 20
    weight: float
    email: EmailStr
    myurl: AnyUrl

In [ ]:
d = {'name': 'mahmood', 'age': 25, 'weight': 75.3, 'email': 'smahmood@test.com', 'myurl': 'http://google.com'}
# d = {'name': 'smahmood', 'age': 35, 'weight': 75.3, 'email': 'smahmoodtest.com'}

In [ ]:
r1 = person(**d)

In [ ]:
r1

person(name='mahmood', age=25, weight=75.3, email='smahmood@test.com', myurl=AnyUrl('http://google.com/'))

In [ ]:
r1.myurl

AnyUrl('http://google.com/')

In [ ]:
r1.model_dump()

{'name': 'mahmood',
 'age': 25,
 'weight': 75.3,
 'email': 'smahmood@test.com',
 'myurl': AnyUrl('http://google.com/')}

In [ ]:
r1.model_dump_json()

'{"name":"mahmood","age":25,"weight":75.3,"email":"smahmood@test.com","myurl":"http://google.com/"}'

In [ ]:
class Patient1(BaseModel):
    name: Annotated[str, Field(max_length=50, title='Name of the patient', description='Give the name of the patient in less than 50 chars', examples=['mahmood', 'srinivas'])]
    email: EmailStr
    myurl: AnyUrl
    age: int = Field(gt=0, lt=120)
    weight: Annotated[float, Field(gt=0, strict=True)]
    married: Annotated[bool, Field(default=None, description='Is the patient married or not')]
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)]
    contact_details: Dict[str, str]


In [ ]:
patient_info = {'name':'smahmood', 'email':'abc@gmail.com', 'linkedin_url':'http://linkedin.com/xyz', 'age': '30', 'weight': 75.2,'contact_details':{'phone':'2353462'}}

In [ ]:
patient_info

{'name': 'smahmood',
 'email': 'abc@gmail.com',
 'linkedin_url': 'http://linkedin.com/xyz',
 'age': '30',
 'weight': 75.2,
 'contact_details': {'phone': '2353462'}}

In [ ]:
patient1 = Patient1(**patient_info)

In [ ]:
def update_patient_data(patient: Patient1):
    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

In [ ]:
update_patient_data(patient1)

smahmood
30
None
None
updated


# class methods

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @classmethod
    def from_string(cls, data_str):
        name, age = data_str.split("-")
        return cls(name, int(age))

In [ ]:
p = Person.from_string("Alice, 30")

In [ ]:
print(p.name)
print(p.age)

Alice
30


# Using **field_validator**

In [ ]:
from pydantic import field_validator

In [ ]:
class Patient2(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @field_validator('email')
    @classmethod
    def email_validator(cls, value):

        valid_domains = ['hdfc.com', 'icici.com']
        # abc@gmail.com
        domain_name = value.split('@')[-1]

        if domain_name not in valid_domains:
            raise ValueError('Not a valid domain')

        return value

    @field_validator('name')
    @classmethod
    def transform_name(cls, value):
        return value.upper()

    @field_validator('age', mode='after')
    @classmethod
    def validate_age(cls, value):
        if 0 < value < 100:
            return value
        else:
            raise ValueError('Age should be in between 0 and 100')


In [ ]:
def update_patient_data2(patient: Patient2):
    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

In [ ]:
patient_info2 = {'name':'srinivas', 'email':'abc@icici.com', 'age': '30', 'weight': 75.2, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462'}}

In [ ]:
patient2 = Patient2(**patient_info2) # validation -> type coercion

In [ ]:
patient2

Patient(name='SRINIVAS', email='abc@icici.com', age=30, weight=75.2, married=True, allergies=['pollen', 'dust'], contact_details={'phone': '2353462'})

In [ ]:
update_patient_data2(patient2)

SRINIVAS
30
['pollen', 'dust']
True
updated


# Using **model_validator**

In [ ]:
from pydantic import model_validator

In [ ]:
class Patient3(BaseModel):
    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @model_validator(mode='after')
    def validate_emergency_contact(cls, model):
        if model.age > 60 and 'emergency' not in model.contact_details:
            raise ValueError('Patients older than 60 must have an emergency contact')
        return model


In [ ]:
patient_info3 = {'name':'David', 'email':'abc@icici.com', 'age': '65', 'weight': 75.2, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462', 'emergency':'235236'}}


In [ ]:
patient3 = Patient3(**patient_info3)

In [ ]:
patient3

Patient(name='David', email='abc@icici.com', age=65, weight=75.2, married=True, allergies=['pollen', 'dust'], contact_details={'phone': '2353462', 'emergency': '235236'})

In [ ]:
def update_patient_data3(patient: Patient3):
    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('updated')

In [ ]:
update_patient_data3(patient3)

David
65
['pollen', 'dust']
True
updated


# @**property**

In [ ]:
class Circle:
    def __init__(self, radius):
        self.r = radius

    @property
    def area(self):
        return 3.14159 * (self.r ** 2)


In [ ]:
c = Circle(5)

In [ ]:
c.area

78.53975

# Using **computed_field**

In [ ]:
from pydantic import computed_field

In [ ]:
class Patient4(BaseModel):

    name: str
    email: EmailStr
    age: int
    weight: float # kg
    height: float # mtr
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

    @computed_field
    @property
    def bmi(self) -> float:
        bmi = round(self.weight/(self.height**2),2)
        return bmi


In [ ]:
def update_patient_data4(patient: Patient4):
    print(patient.name)
    print(patient.age)
    print(patient.allergies)
    print(patient.married)
    print('BMI', patient.bmi)
    print('updated')

In [ ]:
patient_info4 = {'name':'nitish', 'email':'abc@icici.com', 'age': '65', 'weight': 75.2, 'height': 1.72, 'married': True, 'allergies': ['pollen', 'dust'], 'contact_details':{'phone':'2353462', 'emergency':'235236'}}

In [ ]:
patient4 = Patient4(**patient_info4)

In [ ]:
update_patient_data4(patient4)

nitish
65
['pollen', 'dust']
True
BMI 25.42
updated


# Nested Models

In [ ]:
class Address(BaseModel):
    city: str
    state: str
    pin: str

In [ ]:
class Patient5(BaseModel):
    name: str
    gender: str
    age: int
    address: Address

In [ ]:
address_dict = {'city': 'banglore', 'state': 'karnataka', 'pin': '560043'}

In [ ]:
addres = Address(**address_dict)

In [ ]:
patient_dict5 = {'name': 'mahmood', 'gender': 'male', 'age': 35, 'address': addres}

In [ ]:
patient5 = Patient5(**patient_dict5)

In [ ]:
patient5.model_dump()

{'name': 'mahmood',
 'gender': 'male',
 'age': 35,
 'address': {'city': 'banglore', 'state': 'karnataka', 'pin': '560043'}}

In [ ]:
temp = patient5.model_dump()

In [ ]:
temp

{'name': 'mahmood',
 'gender': 'male',
 'age': 35,
 'address': {'city': 'banglore', 'state': 'karnataka', 'pin': '560043'}}

In [ ]:
type(temp)

dict

In [ ]:
temp['address']['city']

'gurgaon'

In [ ]:
newt = patient5.model_dump_json()

In [ ]:
type(newt)

str

In [ ]:
schemas = [
    ResponseSchema(name="answer", description="Direct answer"),
    ResponseSchema(name="confidence", description="Confidence score 0-1")
]

In [ ]:
parser = StructuredOutputParser.from_response_schemas(schemas)

In [ ]:

prompt = ChatPromptTemplate.from_template("""
Answer the question.

{format_instructions}

Question: {question}
""")


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:


chain = prompt | llm | parser

result = chain.invoke({
    "question": "What is LangChain?",
    "format_instructions": parser.get_format_instructions()
})

In [ ]:
print(result)